In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///desktop/MSU_Data_bootcamp/Climate_w_sqlalchemy/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model

Base = automap_base()

#this line reflects the database so that we can examine it
Base.prepare(engine, reflect=True)

#This line gets us the names of tables. 
Base.classes.keys()

['measurement', 'station']

In [7]:
# I am going to assign classes 'measurement' and 'station' to variables
# a database table is always associated with a class in python, which dictates what
#column values are expected. These columns are the attributes of a class
measur = Base.classes.measurement

stat = Base.classes.station

In [8]:
#create a session so that I can connect with the database and retreive data

sesh = Session(engine)

In [10]:
#I want to view the attributes for each table now
#this cell is for the measurements table
fr_measur = sesh.query(measur).first()

fr_measur.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x15712096d30>,
 'prcp': 0.08,
 'station': 'USC00519397',
 'id': 1,
 'tobs': 65.0,
 'date': '2010-01-01'}

In [11]:
#same thing as above for the stations table
fr_stat = sesh.query(stat).first()

fr_stat.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x15711484190>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'latitude': 21.2716,
 'elevation': 3.0,
 'id': 1,
 'station': 'USC00519397'}

In [16]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Last date in file found to be 8/23/17 by inspecting .csv

#set a reference date 12mos before the last date in the set
ref_date = dt.datetime(2016, 8, 23)

#a list of columns to select
sel = [measur.prcp, measur.station, measur.id, measur.tobs, measur.date]

#write a query with a filter that specifies the date be later than the reference date
yr_precip = sesh.query(*sel).filter( measur.date > ref_date).all()

yr_precip                                       


[(0.08, 'USC00519397', 2365, 79.0, '2016-08-24'),
 (0.08, 'USC00519397', 2366, 80.0, '2016-08-25'),
 (0.0, 'USC00519397', 2367, 79.0, '2016-08-26'),
 (0.0, 'USC00519397', 2368, 77.0, '2016-08-27'),
 (0.01, 'USC00519397', 2369, 78.0, '2016-08-28'),
 (0.0, 'USC00519397', 2370, 78.0, '2016-08-29'),
 (0.0, 'USC00519397', 2371, 79.0, '2016-08-30'),
 (0.13, 'USC00519397', 2372, 80.0, '2016-08-31'),
 (0.0, 'USC00519397', 2373, 81.0, '2016-09-01'),
 (0.0, 'USC00519397', 2374, 80.0, '2016-09-02'),
 (0.0, 'USC00519397', 2375, 79.0, '2016-09-03'),
 (0.03, 'USC00519397', 2376, 75.0, '2016-09-04'),
 (None, 'USC00519397', 2377, 79.0, '2016-09-05'),
 (None, 'USC00519397', 2378, 76.0, '2016-09-06'),
 (0.05, 'USC00519397', 2379, 76.0, '2016-09-07'),
 (0.0, 'USC00519397', 2380, 80.0, '2016-09-08'),
 (0.03, 'USC00519397', 2381, 79.0, '2016-09-09'),
 (0.0, 'USC00519397', 2382, 78.0, '2016-09-10'),
 (0.05, 'USC00519397', 2383, 76.0, '2016-09-11'),
 (0.0, 'USC00519397', 2384, 78.0, '2016-09-12'),
 (0.02, 'U

In [21]:
#set up lists for the 4 most interesting quantities in the list above
precip = []
date = []
stid = []
temps = []

#this loop is to sort the quantities into the list
#it also contains a conditional to clean the precipitation data of values that say "none"
#its somewhat ambiguous, but I understand that value to mean "no precipitation", and not "null", so I'm replacing it with 0.0


for y in yr_precip:
    date.append(y[4])
    temps.append(y[3])
    stid.append(y[1])
    if type(y[0]) == float: 
        precip.append(y[0])
    else:
        precip.append(0.0)

        
#writing a dataframe for this data
y_prec = {"Date":date, "Precipitation":precip, "Temperature":temps, "Station_id":stid}
y_prec_df = pd.DataFrame(data = y_prec, index= date)

#sort the dataframe
y_prec_df = y_prec_df.sort_values(by=['Date'])

#display the first few values
y_prec_df.head()

,Date,Precipitation,Temperature,Station_id
2016-08-24,2016-08-24,0.08,79.0,USC00519397
2016-08-24,2016-08-24,0.00,78.0,USC00517948
2016-08-24,2016-08-24,2.15,77.0,USC00519281
2016-08-24,2016-08-24,2.28,80.0,USC00514830
2016-08-24,2016-08-24,2.15,76.0,USC00513117
